# Projeto IC

Download e import das bibliotecas necessárias para a execução das rotinas em python

In [1]:
!pip install Pillow h5py
import time
import matplotlib.pyplot as plt
import numpy as np
import re
import h5py


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


Código construído na primeira fase da IC

O código abaixo realiza a leitura de um arquivo .mrst gerado pelo software GMSH, extrai os dados, as entidades e os elementos.

In [2]:
class MSHFileParse:
    def __init__ (self, filename):
        #buffer
        with open(filename, 'r') as file:
            buffer = file.read()

        # minimal sections that should be present
        self.meshFormat = MeshFormat(buffer)
        self.nodes = Nodes(buffer)
        self.elements = Elements(buffer)
        self.entities = Entities(buffer)


class MeshFormat:
    def __init__ (self, buffer):
        self.version = None
        self.fileType = None
        self.dataSize = None

        self.extractMeshFormat(buffer)
        self.to_hdf5('mesh_format.h5')

    def extractMeshFormat (self, buffer):
        match = re.search(r'\$MeshFormat\s+([\d.]+)\s+(\d+)\s+(\d+)\s+\$EndMeshFormat', buffer)
        if match:
            self.version = float(match.group(1))
            self.fileType = int(match.group(2))
            self.dataSize = int(match.group(3))

            # delete the mesh format section on buffer
            endIndex = buffer.find('$EndMeshFormat') + len('$EndMeshFormat')
            buffer = buffer[endIndex:]
            # print(buffer)
        else:
            raise Exception('MeshFormat not found or invalid')

    def to_hdf5(self, filename):
        with h5py.File(filename, 'w') as file:
            file.create_dataset('version', data=self.version)
            file.create_dataset('fileType', data=self.fileType)
            file.create_dataset('dataSize', data=self.dataSize)

        print(f'Mesh format saved to {filename}')

class Entities:
    def __init__ (self, buffer):
        self.numEntityBlocksPoints = None
        self.numEntityBlocksCurves = None
        self.numEntityBlocksSurfaces = None
        self.numEntityBlocksVolumes = None
        self.entityPoints = []
        self.entityCurves = []
        self.entitySurfaces = []
        self.entityVolumes = []

        self.extractEntities(buffer)
        self.toHdf5('entities.h5')

    def extractEntities(self, buffer):
        match = re.search(r'\$Entities\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s', buffer)
        if match:
            self.numEntityBlocksPoints = int(match.group(1))
            self.numEntityBlocksCurves = int(match.group(2))
            self.numEntityBlocksSurfaces = int(match.group(3))
            self.numEntityBlocksVolumes = int(match.group(4))
            buffer = buffer[match.end():]
            # print(buffer)

            # loop over points
            for _ in range(self.numEntityBlocksPoints):
                match = re.search(r'(\d+)\s(.+)\s', buffer)
                if match:
                    entity_point = {
                        'pointTag': int(match.group(1)),
                        'pointCoord': [float(match.group(2).split()[i]) for i in range(3)],
                        'numPhysicalTags': int(match.group(2).split()[3]),
                        'physicalTags': []
                    }

                    # loop over physical tags
                    for _ in range(entity_point['numPhysicalTags']):
                        entity_point['physicalTags'].append(int(match.group(2).split()[4 + _]))

                    self.entityPoints.append(entity_point)
                    buffer = buffer[match.end():]

            # loop over curves
            for _ in range(self.numEntityBlocksCurves):
                match = re.search(r'(\d+)\s(.+)\s', buffer)
                if match:
                    entity_curve = {
                        'curveTag': int(match.group(1)),
                        'curveCoords': [float(match.group(2).split()[i]) for i in range(6)],
                        'numPhysicalTags': int(match.group(2).split()[6]),
                        'physicalTags': []
                    }

                    # loop over physical tags
                    for _ in range(entity_curve['numPhysicalTags']):
                        entity_curve['physicalTags'].append(int(match.group(2).split()[7 + _]))

                    entity_curve['numBoundingPoints'] = int(match.group(2).split()[7 + entity_curve['numPhysicalTags']])
                    entity_curve['PointTag'] = []

                    # loop over bounding points
                    for _ in range(entity_curve['numBoundingPoints']):
                        entity_curve['PointTag'].append(int(match.group(2).split()[8 + entity_curve['numPhysicalTags'] + _]))

                    self.entityCurves.append(entity_curve)
                    buffer = buffer[match.end():]

            # loop over surfaces
            for _ in range(self.numEntityBlocksSurfaces):
                match = re.search(r'(\d+)\s(.+)\s', buffer)
                if match:
                    entity_surface = {
                        'surfaceTag': int(match.group(1)),
                        'surfaceCoords': [float(match.group(2).split()[i]) for i in range(6)],
                        'numPhysicalTags': int(match.group(2).split()[6]),
                        'physicalTags': []
                    }

                    # loop over physical tags
                    for i in range(entity_surface['numPhysicalTags']):
                        entity_surface['physicalTags'].append(int(match.group(2).split()[7 + i]))

                    entity_surface['numBoundingCurves'] = int(match.group(2).split()[7 + entity_surface['numPhysicalTags']])
                    entity_surface['CurveTag'] = []

                    # loop over bounding curves
                    for _ in range(entity_surface['numBoundingCurves']):
                        entity_surface['CurveTag'].append(int(match.group(2).split()[8 + entity_surface['numPhysicalTags'] + _]))

                    self.entitySurfaces.append(entity_surface)
                    buffer = buffer[match.end():]

            # loop over volumes
            for _ in range(self.numEntityBlocksVolumes):
                match = re.search(r'(\d+)\s(.+)\s', buffer)
                if match:
                    entity_volume = {
                        'volumeTag': int(match.group(1)),
                        'volumeCoords': [float(match.group(2).split()[i]) for i in range(6)],
                        'numPhysicalTags': int(match.group(2).split()[6]),
                        'physicalTags': []
                    }

                    # loop over physical tags
                    for _ in range(entity_volume['numPhysicalTags']):
                        entity_volume['physicalTags'].append(int(match.group(2).split()[7 + _]))

                    entity_volume['numBoundingSurfaces'] = int(match.group(2).split()[7 + entity_volume['numPhysicalTags']])
                    entity_volume['SurfaceTag'] = []

                    # loop over bounding surfaces
                    for _ in range(entity_volume['numBoundingSurfaces']):
                        entity_volume['SurfaceTag'].append(int(match.group(2).split()[8 + entity_volume['numPhysicalTags'] + _]))

                    self.entityVolumes.append(entity_volume)
                    buffer = buffer[match.end():]



        else:
            raise Exception('Entities not found or invalid')

    def toNumpy(self):
        #points
        pointsTags = [np.array(block['physicalTags']) for block in self.entityPoints]

        #curves
        curvesTags = [np.array(block['physicalTags']) for block in self.entityCurves]
        curvesBoundingPoints = [np.array(block['PointTag']) for block in self.entityCurves]

        #surfaces
        surfacesTags = [np.array(block['physicalTags']) for block in self.entitySurfaces]
        surfacesCurves = [np.array(block['CurveTag']) for block in self.entitySurfaces]

        #volumes
        volumesTags = [np.array(block['physicalTags']) for block in self.entityVolumes]
        volumesSurfaces = [np.array(block['SurfaceTag']) for block in self.entityVolumes]

        return pointsTags, (curvesTags, curvesBoundingPoints), (surfacesTags, surfacesCurves), (volumesTags, volumesSurfaces)


    def toHdf5(self, filename):
        pointsTags, (curvesTags, curvesBoundingPoints), (surfacesTags, surfacesCurves), (volumesTags, volumesSurfaces) = self.toNumpy()

        with h5py.File(filename, 'w') as file:
            # Saving general information
            file.create_dataset('numEntityBlocksPoints', data=self.numEntityBlocksPoints)
            file.create_dataset('numEntityBlocksCurves', data=self.numEntityBlocksCurves)
            file.create_dataset('numEntityBlocksSurfaces', data=self.numEntityBlocksSurfaces)
            file.create_dataset('numEntityBlocksVolumes', data=self.numEntityBlocksVolumes)

            # Saving entity blocks points
            for i, (tags) in enumerate(zip(pointsTags)):
                group = file.create_group(f'entityBlockPoints{i}')
                group.create_dataset('pointTag', data=self.entityPoints[i]['pointTag'])
                group.create_dataset('pointCoord', data=self.entityPoints[i]['pointCoord'])
                group.create_dataset('numPhysicalTags', data=self.entityPoints[i]['numPhysicalTags'])
                group.create_dataset('physicalTags', data=tags)

            # Saving entity blocks curves
            for i, (tags, boundingPoints) in enumerate(zip(curvesTags, curvesBoundingPoints)):
                group = file.create_group(f'entityBlockCurves{i}')
                group.create_dataset('curveTag', data=self.entityCurves[i]['curveTag'])
                group.create_dataset('curveCoords', data=self.entityCurves[i]['curveCoords'])
                group.create_dataset('numPhysicalTags', data=self.entityCurves[i]['numPhysicalTags'])
                group.create_dataset('physicalTags', data=tags)
                group.create_dataset('numBoundingPoints', data=self.entityCurves[i]['numBoundingPoints'])
                group.create_dataset('pointTag', data=boundingPoints)

            # Saving entity blocks surfaces
            for i, (tags, curves) in enumerate(zip(surfacesTags, surfacesCurves)):
                group = file.create_group(f'entityBlockSurfaces{i}')
                group.create_dataset('surfaceTag', data=self.entitySurfaces[i]['surfaceTag'])
                group.create_dataset('surfaceCoords', data=self.entitySurfaces[i]['surfaceCoords'])
                group.create_dataset('numPhysicalTags', data=self.entitySurfaces[i]['numPhysicalTags'])
                group.create_dataset('physicalTags', data=tags)
                group.create_dataset('numBoundingCurves', data=self.entitySurfaces[i]['numBoundingCurves'])
                group.create_dataset('curveTag', data=curves)

            # Saving entity blocks volumes
            for i, (tags, surfaces) in enumerate(zip(volumesTags, volumesSurfaces)):
                group = file.create_group(f'entityBlockVolumes{i}')
                group.create_dataset('volumeTag', data=self.entityVolumes[i]['volumeTag'])
                group.create_dataset('volumeCoords', data=self.entityVolumes[i]['volumeCoords'])
                group.create_dataset('numPhysicalTags', data=self.entityVolumes[i]['numPhysicalTags'])
                group.create_dataset('physicalTags', data=tags)
                group.create_dataset('numBoundingSurfaces', data=self.entityVolumes[i]['numBoundingSurfaces'])
                group.create_dataset('surfaceTag', data=surfaces)

        print(f'Entities saved to {filename}')


class Nodes:
    def __init__ (self, buffer):
        self.numEntityBlocks = None
        self.numNodes = None
        self.minNodeTag = None
        self.maxNodeTag = None
        self.entityBlock = []

        self.extractNodes(buffer)
        self.toHdf5('nodes.h5')

    def extractNodes(self, buffer):
        match = re.search(r'\$Nodes\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s', buffer)
        if match:
            # general information
            self.numEntityBlocks = int(match.group(1))
            self.numNodes = int(match.group(2))
            self.minNodeTag = int(match.group(3))
            self.maxNodeTag = int(match.group(4))
            buffer = buffer[match.end():]
            # print(buffer)

            # loop over entity blocks
            for _ in range(self.numEntityBlocks):
                match = re.search(r'(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s', buffer)
                if match:
                    entity_block = {
                        'entityDim': int(match.group(1)),
                        'entityTag': int(match.group(2)),
                        'parametric': int(match.group(3)),
                        'numNodesInBlock': int(match.group(4)),
                        'nodeTag': [],
                        'nodeCoord': []
                    }
                    buffer = buffer[match.end():]

                    # loop over nodes in block
                    # fisrt all the tags
                    for _ in range(entity_block['numNodesInBlock']):
                        match = re.search(r'(\d+)\s', buffer)
                        if match:
                            node_tag = int(match.group(1))
                            entity_block['nodeTag'].append(node_tag)
                            buffer = buffer[match.end():]

                    # then all the coordinates
                    for _ in range(entity_block['numNodesInBlock']):
                        match = re.search(r'(.+)\s', buffer)
                        if match:
                            node_coord = [float(coord) for coord in match.group(1).split()]
                            entity_block['nodeCoord'].append(node_coord)
                            buffer = buffer[match.end():]
                        else:
                            raise Exception('Node coordinates not found or invalid')

                    self.entityBlock.append(entity_block)

        else:
            raise Exception('Nodes not found or invalid')

    def toNumpy(self):
        nodeTags = [np.array(block['nodeTag']) for block in self.entityBlock]
        nodeCoords = [np.array(block['nodeCoord']) for block in self.entityBlock]

        return nodeTags, nodeCoords

    def toHdf5(self, filename):
        nodeTags, nodeCoords = self.toNumpy()

        with h5py.File(filename, 'w') as file:
            # Saving general information
            file.create_dataset('numEntityBlocks', data=self.numEntityBlocks)
            file.create_dataset('numNodes', data=self.numNodes)
            file.create_dataset('minNodeTag', data=self.minNodeTag)
            file.create_dataset('maxNodeTag', data=self.maxNodeTag)

            # Saving entity blocks
            for i, (tags, coords) in enumerate(zip(nodeTags, nodeCoords)):
                group = file.create_group(f'entityBlock{i}')
                group.create_dataset('entityDim', data=self.entityBlock[i]['entityDim'])
                group.create_dataset('entityTag', data=self.entityBlock[i]['entityTag'])
                group.create_dataset('parametric', data=self.entityBlock[i]['parametric'])
                group.create_dataset('numNodesInBlock', data=self.entityBlock[i]['numNodesInBlock'])
                group.create_dataset('nodeTags', data=tags)
                group.create_dataset('nodeCoords', data=coords)

        print(f'Nodes saved to {filename}')


class Elements:
    def __init__(self, buffer):
        self.numEntityBlocks = None
        self.numElements = None
        self.minElementTag = None
        self.maxElementTag = None
        self.entityBlock = []

        self.extractElements(buffer)
        self.toHdf5('elements.h5')

    def extractElements(self, buffer):
        match = re.search(r'\$Elements\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s', buffer)
        if match:
            # general information (first line)
            self.numEntityBlocks = int(match.group(1))
            self.numElements = int(match.group(2))
            self.minElementTag = int(match.group(3))
            self.maxElementTag = int(match.group(4))
            buffer = buffer[match.end():]

            # loop over entity blocks
            for _ in range(self.numEntityBlocks):
                match = re.search(r'(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s', buffer)
                if match:
                    entity_block = {
                        'entityDim': int(match.group(1)),
                        'entityTag': int(match.group(2)),
                        'elementType': int(match.group(3)),
                        'numElementsInBlock': int(match.group(4)),
                        'elementTags': [],
                        'elementNodes': []
                    }
                    buffer = buffer[match.end():]

                    # loop over elements in block
                    for _ in range(entity_block['numElementsInBlock']):
                        match = re.search(r'(\d+)\s(.+)\s', buffer)
                        if match:
                            element_tag = int(match.group(1))
                            element_nodes = [int(node) for node in match.group(2).split()]
                            entity_block['elementTags'].append(element_tag)
                            entity_block['elementNodes'].append(element_nodes)
                            buffer = buffer[match.end():]

                    self.entityBlock.append(entity_block)
        else:
            raise Exception('Elements not found or invalid')

    def toNumpy(self):
        elementTags = [np.array(block['elementTags']) for block in self.entityBlock]
        elementNodes = [np.array(block['elementNodes']) for block in self.entityBlock]

        return elementTags, elementNodes

    def toHdf5(self, filename):
        elementTags, elementNodes = self.toNumpy()

        with h5py.File(filename, 'w') as file:
            # Saving general information
            file.create_dataset('numEntityBlocks', data=self.numEntityBlocks)
            file.create_dataset('numElements', data=self.numElements)
            file.create_dataset('minElementTag', data=self.minElementTag)
            file.create_dataset('maxElementTag', data=self.maxElementTag)

            # Saving entity blocks
            for i, (tags, nodes) in enumerate(zip(elementTags, elementNodes)):
                group = file.create_group(f'entityBlock{i}')
                group.create_dataset('entityDim', data=self.entityBlock[i]['entityDim'])
                group.create_dataset('entityTag', data=self.entityBlock[i]['entityTag'])
                group.create_dataset('elementType', data=self.entityBlock[i]['elementType'])
                group.create_dataset('numElementsInBlock', data=self.entityBlock[i]['numElementsInBlock'])
                group.create_dataset('elementTags', data=tags)
                group.create_dataset('elementNodes', data=nodes)

        print(f'Elements saved to {filename}')

In [3]:
class MeshDataStructure:
  def __init__(self, nodes_file: str, connectivities_file: str):
    self.nodes_file = nodes_file
    self.connectivities_file = connectivities_file
    self.nodes = None
    self.connectivities = None
    self.centroids = None
    self.coords_centroids = None
    self.nHorizontal = None
    self.nVertical = None
    self.coord_grossa = None
    self.grossa_centroids_array = None
    self.vertices = None
    self.arestas = None
    self.faces = None
    self.nos_na_proxima_hierarquia = None
    self.conexoes_na_proxima_hierarquia = None

  def set_nodes(self):
    nos = np.array([[1, 2, 3]])
    with h5py.File(self.nodes_file, 'r') as file:
        grupos = []
        for group in file.keys():
            if("entity" in group):
              grupos.append(group)

        for grupo_nome in grupos:
          grupo = file[grupo_nome]
          for dataset in grupo.keys():
              dados = []
              try:
                dados = grupo[dataset][:]
                dados = np.round(dados, 6)
                nos = np.concatenate((nos, dados))
              except:
                dados = grupo[dataset]
        nos = nos[1:]
        self.nodes = nos

  def set_connectivities(self):
    conexao = np.array([[1,2,3,4]])
    with h5py.File(self.connectivities_file, 'r') as file:
        grupos = []
        for group in file.keys():
            if("entity" in group):
              grupos.append(group)

        for grupo_nome in grupos:
          grupo = file[grupo_nome]
          for dataset in grupo.keys():
              dados = []
              try:
                dados = grupo[dataset][:]
                if(isinstance(dados[0], np.ndarray)):
                  if(len(dados[0]) == 4):
                    conexao = np.concatenate((conexao, dados))
              except:
                dados = grupo[dataset]
        conexao = conexao[1:]
        self.connectivities = conexao

  def get_centroids(self):
    connectivities = np.array(self.connectivities) - 1  
    nodes = np.array(self.nodes)

    coords = nodes[connectivities]
    
    sum_coords = np.sum(coords, axis=1)
    centroids = sum_coords / 4.0
    
    centroids_rounded = np.round(centroids, 6)
    
    indices = np.arange(len(centroids)).reshape(-1, 1)
    centroids_with_indices = np.hstack((centroids_rounded, indices))
    
    self.centroids = centroids_with_indices
    self.coords_centroids = coords

  def sort_centroids(self, centroids):
    centroids = np.asarray(centroids)
    indices_ordenados = np.lexsort((centroids[:, 1], centroids[:, 0]))
    return centroids[indices_ordenados]


  def get_dimensions(self):
    error = 0.0005
    firstX = self.centroids[0, 0]
    firstY = self.centroids[0, 1]

    nHorizontal = np.sum((self.centroids[:, 0] >= firstX - error) & (self.centroids[:, 0] <= firstX + error))

    nVertical = np.sum((self.centroids[:, 1] >= firstY - error) & (self.centroids[:, 1] <= firstY + error))

    self.nHorizontal = nHorizontal
    self.nVertical = nVertical

  def divide_mesh(self, centroids, H, V, nX, nY):
    if((H % nX != 0) or (V % nY) != 0):
      print("Os valores H e V não são múltiplos de nX e nY")
      return

    coord_grossa = np.empty(centroids.shape[0], dtype=object)

    grossa_centroids_array = np.empty((H // nX) * (V // nY), dtype=object)
    grossa_centroids_array[:] = [np.array([], dtype=int) for _ in range((H // nX) * (V // nY))]

    vertices = np.zeros((H // nX) * (V // nY), dtype=int)
    
    arestas = np.empty((H // nX) * (V // nY), dtype=object)
    faces = np.empty((H // nX) * (V // nY), dtype=object)
    
    arestas[:] = [np.array([], dtype=int) for _ in range((H // nX) * (V // nY))]
    faces[:] = [np.array([], dtype=int) for _ in range((H // nX) * (V // nY))]

    ja_foi_mapeado = set()

    # Cria uma grade de valores de i e j
    i_vals = np.arange(H)
    j_vals = np.arange(V)
    i_grid, j_grid = np.meshgrid(i_vals, j_vals, indexing='ij')
    
    # Calcula indX e indY vetorizados
    indX = i_grid // nX
    indY = j_grid // nY
    
    # Calcula os nós
    nodes = i_grid * V + j_grid
    
    # Atribui a coordenada grossa para cada nó
    coord_grossa[nodes.flatten()] = list(zip(indX.flatten(), indY.flatten()))
    
    # Prepara grossa_centroids_array usando uma lista de compreensão
    grossa_centroids_array = np.empty((H // nX) * (V // nY), dtype=object)
    grossa_centroids_array[:] = [np.array([], dtype=int) for _ in range((H // nX) * (V // nY))]
    
    # Atualiza grossa_centroids_array com os nós, de forma vetorizada
    for indX_val, indY_val, node in zip(indX.flatten(), indY.flatten(), nodes.flatten()):
        idx = indX_val * (V // nY) + indY_val
        grossa_centroids_array[idx] = np.append(grossa_centroids_array[idx], node)
          
    # Calcula x e y para cada índice
    indices = np.arange((H // nX) * (V // nY))
    x = indices // (V // nY)
    y = indices % (V // nY)
    
    # Calcula os vértices para cada condição
    vertices[(x == 0) & (y == 0)] = [grossa_centroids_array[i][0] for i in indices[(x == 0) & (y == 0)]]
    vertices[(x == (H // nX - 1)) & (y == 0)] = [grossa_centroids_array[i][-1] for i in indices[(x == (H // nX - 1)) & (y == 0)]]
    vertices[(x == 0) & (y == (V // nY - 1))] = [grossa_centroids_array[i][-nY] for i in indices[(x == 0) & (y == (V // nY - 1))]]
    vertices[(x == (H // nX - 1)) & (y == (V // nY - 1))] = [grossa_centroids_array[i][-1] for i in indices[(x == (H // nX - 1)) & (y == (V // nY - 1))]]
    vertices[(x == 0) & (y > 0) & (y < (V // nY - 1))] = [grossa_centroids_array[i][nY // 2] for i in indices[(x == 0) & (y > 0) & (y < (V // nY - 1))]]
    vertices[(y == (V // nY - 1)) & (x > 0) & (x < (H // nX - 1))] = [grossa_centroids_array[i][nY * (nX // 2 + 1) - 1] for i in indices[(y == (V // nY - 1)) & (x > 0) & (x < (H // nX - 1))]]
    vertices[(y == 0) & (x > 0) & (x < (H // nX - 1))] = [grossa_centroids_array[i][nY * (nX // 2)] for i in indices[(y == 0) & (x > 0) & (x < (H // nX - 1))]]
    vertices[(x == (H // nX - 1)) & (y > 0) & (y < (V // nY - 1))] = [grossa_centroids_array[i][-2] for i in indices[(x == (H // nX - 1)) & (y > 0) & (y < (V // nY - 1))]]
    vertices[(x > 0) & (x < (H // nX - 1)) & (y > 0) & (y < (V // nY - 1))] = [grossa_centroids_array[i][(nX * nY) // 2] for i in indices[(x > 0) & (x < (H // nX - 1)) & (y > 0) & (y < (V // nY - 1))]]
    
    # Adiciona valores ao conjunto
    ja_foi_mapeado = set(vertices)

    # Preciso pegar as arestas
    # Borda lateral esquerda
    for index in range(0, V):
      if index in ja_foi_mapeado:
        continue
      ja_foi_mapeado.add(index)
      pertence_na_grossa = coord_grossa[index]
      index_na_grossa = pertence_na_grossa[0]*((V // nY)) + pertence_na_grossa[1]
      arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([index])))


    # Borda inferior
    for index in range(0, H*V, V):
      if index in ja_foi_mapeado:
        continue
      ja_foi_mapeado.add(index)
      pertence_na_grossa = coord_grossa[index]
      index_na_grossa = pertence_na_grossa[0]*((V // nY)) + pertence_na_grossa[1]
      arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([index])))


    # Borda superior
    for index in range(V-1, H*V, V):
      if index in ja_foi_mapeado:
        continue
      ja_foi_mapeado.add(index)
      pertence_na_grossa = coord_grossa[index]
      index_na_grossa = pertence_na_grossa[0]*((V // nY)) + pertence_na_grossa[1]
      arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([index])))
     

    # Borda lateral direita
    for index in range(V*(H-1), H*V):
      if index in ja_foi_mapeado:
        continue
      ja_foi_mapeado.add(index)
      pertence_na_grossa = coord_grossa[index]
      index_na_grossa = pertence_na_grossa[0]*((V // nY)) + pertence_na_grossa[1]
      arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([index])))

    
    #Para cada vertice não-borda, só andar pros 4 lados até atingir um vértice.

    todos_vertices = vertices
    todos_vertices_nao_borda = []

    for index in range(0, (H // nX) * (V // nY)):
      i = index // ((V // nY))
      j = index % ((V // nY))
      if(i == 0 or i == (H//nX - 1) or j == 0 or j == (V // nY - 1)):
        continue
      todos_vertices_nao_borda.append(vertices[index])

    todos_vertices_nao_borda = np.array(todos_vertices_nao_borda)
        

    for index in todos_vertices_nao_borda:
      # Ando para cima até encontrar um vertice.
      daVezU = index + 1

      while (daVezU not in ja_foi_mapeado):
        pertence_na_grossa = coord_grossa[daVezU]
        index_na_grossa = pertence_na_grossa[0]*((V // nY)) + pertence_na_grossa[1]
        arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([daVezU])))
        ja_foi_mapeado.add(daVezU)
        daVezU += 1

      # Ando para baixo
      daVezD = index - 1
      while (daVezD not in ja_foi_mapeado):
        pertence_na_grossa = coord_grossa[daVezD]
        index_na_grossa = pertence_na_grossa[0]*((V // nY)) + pertence_na_grossa[1]
        arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([daVezD])))
        ja_foi_mapeado.add(daVezD)
        daVezD -= 1

      # Ando para a direita
      daVezR = index + V
      while (daVezR not in ja_foi_mapeado):
        pertence_na_grossa = coord_grossa[daVezR]
        index_na_grossa = pertence_na_grossa[0]*((V // nY)) + pertence_na_grossa[1]
        arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([daVezR])))
        ja_foi_mapeado.add(daVezR)
        daVezR += V

      # Ando para a esquerda
      daVezL = index - V
      while (daVezL not in ja_foi_mapeado):
        pertence_na_grossa = coord_grossa[daVezL]
        index_na_grossa = pertence_na_grossa[0]*((V // nY)) + pertence_na_grossa[1]
        arestas[index_na_grossa] = np.concatenate((arestas[index_na_grossa], np.array([daVezL])))
        ja_foi_mapeado.add(daVezL)
        daVezL -= V

    for index in range(0, V*H):
      if(index not in ja_foi_mapeado):
        ja_foi_mapeado.add(index)
        pertence_na_grossa = coord_grossa[index]
        index_na_grossa = pertence_na_grossa[0]*((V // nY)) + pertence_na_grossa[1]
        faces[index_na_grossa] = np.concatenate((faces[index_na_grossa], np.array([index])))

    self.coord_grossa = coord_grossa
    self.grossa_centroids_array = grossa_centroids_array
    self.vertices = vertices
    self.arestas = arestas
    self.faces = faces
    self.nX = nX
    self.nY = nY

  def proxima_hierarquia(self):
    sz = len(self.coord_grossa)
    
    indices = np.arange(sz)
    coord_grossa_array = np.array(self.coord_grossa)
    
    chaves_unicas = np.unique(coord_grossa_array)
    num_chaves = len(chaves_unicas)
    
    nos_por_index_grossa = [np.array([]) for _ in range(num_chaves)]
    
    for i, chave in enumerate(chaves_unicas):
        chave_array = np.array(chave, dtype=object)  # Garante que a chave seja tratada como array NumPy
        indices_correspondentes = np.where(coord_grossa_array == chave_array)[0]
        nos_por_index_grossa[i] = indices_correspondentes
    
    nos_na_proxima_hierarquia = np.array([[1, 2, 3]])
    conexoes_na_proxima_hierarquia = np.array([[1, 2, 3, 4]])
    index = 1
    
    # Processa cada grupo de índices
    for elto in nos_por_index_grossa:
        tam = len(elto)
        if tam < self.nY:
            continue  # Evita processamento se não houver índices suficientes

        esquerda_inferior = elto[0]
        esquerda_superior = elto[self.nY - 1]
        direita_inferior = elto[tam - self.nY]
        direita_superior = elto[tam - 1]

        self.coords_centroids[esquerda_inferior] = self.sort_centroids(self.coords_centroids[esquerda_inferior])
        self.coords_centroids[esquerda_superior] = self.sort_centroids(self.coords_centroids[esquerda_superior])
        self.coords_centroids[direita_inferior] = self.sort_centroids(self.coords_centroids[direita_inferior])
        self.coords_centroids[direita_superior] = self.sort_centroids(self.coords_centroids[direita_superior])

        nos_na_proxima_hierarquia = np.concatenate((nos_na_proxima_hierarquia, [self.coords_centroids[esquerda_inferior][0]]))
        nos_na_proxima_hierarquia = np.concatenate((nos_na_proxima_hierarquia, [self.coords_centroids[esquerda_superior][1]]))
        nos_na_proxima_hierarquia = np.concatenate((nos_na_proxima_hierarquia, [self.coords_centroids[direita_inferior][2]]))
        nos_na_proxima_hierarquia = np.concatenate((nos_na_proxima_hierarquia, [self.coords_centroids[direita_superior][3]]))

        conexoes_na_proxima_hierarquia = np.concatenate((conexoes_na_proxima_hierarquia, [[index, index + 1, index + 3, index + 2]]))
        index += 4

    # Remove o primeiro elemento, que era inicializado
    nos_na_proxima_hierarquia = nos_na_proxima_hierarquia[1:]
    conexoes_na_proxima_hierarquia = conexoes_na_proxima_hierarquia[1:]

    self.nos_na_proxima_hierarquia = nos_na_proxima_hierarquia
    self.conexoes_na_proxima_hierarquia = conexoes_na_proxima_hierarquia

  def plot_mesh(self):

    plt.figure(figsize=(40, 32))
    squares = self.connectivities

    # Print dos nos
    for i, (x, y, z) in enumerate(self.nodes):
        plt.plot(x, y, 'bo')

    # Print das conexões
    for sq in squares:
      p1 = self.nodes[sq[0] - 1]
      p2 = self.nodes[sq[1] - 1]
      p3 = self.nodes[sq[2] - 1]
      p4 = self.nodes[sq[3] - 1]
      plt.plot([p1[0], p2[0], p3[0], p4[0], p1[0]], [p1[1], p2[1], p3[1], p4[1], p1[1]], 'red')

    for i, (x, y, z, index) in enumerate(self.centroids):
      plt.plot(x, y, 'bo')  # Plot do ponto

      # Adicionando o número do nó ao lado do ponto
      plt.text(x, y, str(i), color='black', fontsize=12, ha='center', va='bottom')

    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Plotagem da malha')
    plt.grid(True)
    plt.axis('equal')
    plt.show()

  def process(self, nX, nY, H, V):
    self.set_nodes()
    self.set_connectivities()
    self.get_centroids()
    self.centroids = self.sort_centroids(self.centroids)
    self.nHorizontal = H
    self.nVertical = V
    self.divide_mesh(self.centroids, self.nHorizontal, self.nVertical, nX, nY)
    self.proxima_hierarquia()
